In [2]:
import ABCD_ML

In [3]:
nda_dr = '/mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/'
test_mapping_loc = nda_dr + 'ABCD_Release_ Notes_Data_Release_ 2.0/22. ABCD_Release_2.0_mapping_r.csv'
test_data_loc1 = nda_dr + 'MRI/ABCD dMRI DTI Full Destrieux Parcellation Part 2.csv'
test_data_loc2 = nda_dr + 'MRI/ABCD sMRI Destrieux Parcellation Part 1.csv'
test_scores_loc1 = nda_dr + 'Neurocognition/ABCD Youth NIH TB Summary Scores.csv'
test_exclusion_loc1 = '/home/sage/ABCD_Ev_Search/Data/ABCD_testsetIDs.csv'
test_exclusion_loc2 = '/home/sage/bader_things/invalid_pguids.txt'
test_strat_loc1 = nda_dr + 'Other Non-Imaging/ABCD ACS Post Stratification Weights.csv'
test_strat_loc2 = nda_dr + 'Other Non-Imaging/ABCD Longitudinal Tracking.csv'
test_covar_loc = nda_dr + 'Mental Health/ABCD Parent Demographics Survey.csv'

In [4]:
ML = ABCD_ML.ABCD_ML(eventname = 'baseline_year_1_arm_1',
                     use_default_subject_ids = True,
                     verbose = True
                     )

ML.load_name_mapping(loc = test_mapping_loc)

Loaded map file


In [5]:
ML.load_data(loc = test_data_loc1,
             drop_keys = ['spostcentrallh'],
             filter_outlier_percent = .0001,
             winsorize_val=.001
             )

ML.load_data(loc = test_data_loc2,
             drop_keys = []
             )

Loading /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/MRI/ABCD dMRI DTI Full Destrieux Parcellation Part 2.csv
dropped ['abcd_ddtifp201_id', 'dataset_id'] columns by default
Dropped 6 columns, per drop_keys argument
Dropped 1095 rows for missing values
Dropped rows with missing data
Filtered data for outliers with value:  0.0001
Winsorized data with value:  0.001
loaded shape:  (9156, 835)
Removing non overlapping + excluded subjects
Total subjects =  9156

Loading /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/MRI/ABCD sMRI Destrieux Parcellation Part 1.csv
dropped ['abcd_mrisdp101_id', 'dataset_id'] columns by default
Dropped 0 columns, per drop_keys argument
Dropped 2387 rows for missing values
Dropped rows with missing data
loaded shape:  (9149, 906)
Merged with existing!
Removing non overlapping + excluded subjects
Total subjects =  7272



In [6]:
ML.load_scores(loc=test_covar_loc,
               col_name='demo_sex_v2',
               data_type = 'binary',
              )

Loading  /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/Mental Health/ABCD Parent Demographics Survey.csv
More than two unique score values found, filtered all but [1. 2.]
Final shape:  (11866, 1)
Removing non overlapping + excluded subjects
Total subjects =  7266



In [7]:
ML.load_scores(loc=test_covar_loc,
               col_name='demo_relig_v2',
               data_type = 'categorical',
              )

Loading  /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/Mental Health/ABCD Parent Demographics Survey.csv
Encoded to 17 categories
Final shape:  (11310, 17)
Removing non overlapping + excluded subjects
Total subjects =  6941



In [8]:
#ML.load_scores(loc=test_scores_loc1,
#               col_name='nihtbx_cardsort_agecorrected',
#               data_type = 'float',
#               filter_outlier_percent=(.001, .999)
#              )

In [9]:
ML.load_exclusions(exclusions=['INV0A6WVRZY', 'NDAR_INVZXF5C635'])
#ML.load_exclusions(loc = test_exclusion_loc1)
ML.load_exclusions(loc = test_exclusion_loc2)

Total excluded subjects:  2
Removing non overlapping + excluded subjects
Total subjects =  6939

Total excluded subjects:  1139
Removing non overlapping + excluded subjects
Total subjects =  6224



In [10]:
ML.load_strat_values(loc = test_strat_loc1,
                     col_names = ['race_ethnicity', 'rel_family_id'])

ML.load_strat_values(loc = test_strat_loc2,
                     col_names = ['site_id_l'])

Reading stratification values from /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/Other Non-Imaging/ABCD ACS Post Stratification Weights.csv
Removing non overlapping + excluded subjects
Total subjects =  6214

Reading stratification values from /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/Other Non-Imaging/ABCD Longitudinal Tracking.csv
Merged with existing!
Removing non overlapping + excluded subjects
Total subjects =  6214



In [11]:
ML.load_covars(loc=test_covar_loc,
               col_names = ['demo_ed_v2', 'demo_sex_v2', 'demo_yrs_1'],
               data_types = ['o', 'b', 'c'],
               dummy_code_categorical = True,
               filter_float_outlier_percent = (.001, .999),
               standardize = True,
               normalize = True) 

Reading covariates from /mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/Mental Health/ABCD Parent Demographics Survey.csv
load: demo_ed_v2
load: demo_sex_v2
More than two unique score values found, filtered all but [1. 2.]
load: demo_yrs_1
Encoded to 4 categories
Removing non overlapping + excluded subjects
Total subjects =  6132



In [12]:
#ML.train_test_split(test_size=.2)
#test1 = ML.CV.k_fold(ML.train_subjects, 3, return_index=False)
#test2 = ML.CV.k_fold(ML.train_subjects, 3, return_index=True)

In [13]:
ML.define_validation_strategy(groups='site_id_l')

In [14]:
ML.train_test_split(test_size=.2)

Performed train/test split, train size: 4330 test size:  1802


In [15]:
import numpy as np
X = np.array(ML.data)
y = np.array(ML.scores)

X.shape, y.shape

((6132, 1741), (6132, 17))

In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier, GaussianProcessRegressor
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.linear_model import (LogisticRegressionCV, ElasticNetCV, LinearRegression,
                                  OrthogonalMatchingPursuitCV,LarsCV, RidgeCV)

#from sklearn.svm import SVC, LinearSVR
#from sklearn.neural_network import MLPClassifier

#from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBRegressor
#from lightgbm import LGBMRegressor, LGBMClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [29]:
clf = KNeighborsClassifier(n_jobs=8)

In [35]:
q = {'5': 0}
q.update({})
q

{'5': 0}

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.svm import SVC

In [17]:
X_train, X_test = X[:-1000], X[-1000:]
y_train, y_test = y[:-1000], y[-1000:]

In [18]:
clf = DecisionTreeClassifier()

In [ ]:
#y_train = ML.score_encoder[1].inverse_transform(y_train).squeeze()

In [19]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [ ]:
y_test = ML.score_encoder.inverse_transform(y_test).squeeze()

In [20]:
preds = clf.predict(X_test)
pred_proba = clf.predict_proba(X_test)
probs = np.stack([p[:,1] for p in pred_proba], axis=1)

In [ ]:
matthews_corrcoef(y_test, preds)

In [ ]:
roc_auc_score(y_test, probs, None)

In [ ]:
make_scorer(roc_auc_score, needs_proba=True, average='micro')

In [ ]:
#extra_params = {
#               'rf': {'n_iter': 20,
#                      'verbose': 1},
#               'elastic cv' : {'max_iter': 20000}
#               }

#ML.evaluate_regression_model(
#                            model_type = 'linear',
#                            data_scaler = 'standard',
#                            n_splits = 2,
#                            n_repeats = 1,
#                            int_cv = 2,
#                            extra_params = extra_params
#                            )